In [1]:
!pip install tensorflow


In [14]:

# Import necessary libraries
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import os
import matplotlib.pyplot as plt


In [ ]:
# Load the FruitNet dataset using TensorFlow's ImageDataGenerator class.
train_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(
#         '/path/to/dataset/'
         'C:\\Users\\n\\Downloads\\FruitNetDataset',
        target_size=(256, 256),
        batch_size=32,
        class_mode='categorical')


In [ ]:
# Preprocess the dataset by dividing the images into three categories: good, bad, and mixed.
good_fruits = [image for image in train_generator if np.argmax(image[1]) == 0]
bad_fruits = [image for image in train_generator if np.argmax(image[1]) == 1]
mixed_fruits = [image for image in train_generator if np.argmax(image[1]) == 2]


# Define the generator and discriminator models for the Cycle-GAN algorithm. 
# The generator model  have two components: an encoder and a decoder. The discriminator model is a convolutional neural network.

In [ ]:

 # Define the generator models
# generator_G = keras.Sequential([...])
# generator_F = keras.Sequential([...])

from tensorflow import keras
def build_generator():
    # Generator G: Converts bad quality images to good quality images
    generator_G = keras.Sequential([
        keras.layers.Input(shape=(256, 256, 3)),
        # Downsampling
        keras.layers.Conv2D(64, 4, strides=2, padding='same', activation='relu'),
        keras.layers.BatchNormalization(),
        keras.layers.Conv2D(128, 4, strides=2, padding='same', activation='relu'),
        keras.layers.BatchNormalization(),
        keras.layers.Conv2D(256, 4, strides=2, padding='same', activation='relu'),
        keras.layers.BatchNormalization(),
        # Residual blocks
        keras.layers.Conv2D(256, 3, padding='same', activation='relu'),
        keras.layers.BatchNormalization(),
        keras.layers.Conv2D(256, 3, padding='same', activation=None),
        keras.layers.BatchNormalization(),
        keras.layers.Add(),
        keras.layers.Activation('relu'),
        keras.layers.Conv2D(256, 3, padding='same', activation='relu'),
        keras.layers.BatchNormalization(),
        keras.layers.Conv2D(256, 3, padding='same', activation=None),
        keras.layers.BatchNormalization(),
        keras.layers.Add(),
        keras.layers.Activation('relu'),
        keras.layers.Conv2D(256, 3, padding='same', activation='relu'),
        keras.layers.BatchNormalization(),
        keras.layers.Conv2D(256, 3, padding='same', activation=None),
        keras.layers.BatchNormalization(),
        keras.layers.Add(),
        keras.layers.Activation('relu'),
        keras.layers.Conv2DTranspose(128, 4, strides=2, padding='same', activation='relu'),
        keras.layers.BatchNormalization(),
        keras.layers.Conv2DTranspose(64, 4, strides=2, padding='same', activation='relu'),
        keras.layers.BatchNormalization(),
        keras.layers.Conv2DTranspose(3, 4, strides=2, padding='same', activation='sigmoid')
    ], name='generator_G')

    # Generator F: Converts good quality images to bad quality images
    generator_F = keras.Sequential([
        keras.layers.Input(shape=(256, 256, 3)),
        # Downsampling
        keras.layers.Conv2D(64, 4, strides=2, padding='same', activation='relu'),
        keras.layers.BatchNormalization(),
        keras.layers.Conv2D(128, 4, strides=2, padding='same', activation='relu'),
        keras.layers.BatchNormalization(),
        keras.layers.Conv2D(256, 4, strides=2, padding='same', activation='relu'),
        keras.layers.BatchNormalization(),
        # Residual blocks
        keras.layers.Conv2D(256, 3, padding='same', activation='relu'),
        keras.layers.BatchNormalization(),
        keras.layers.Conv2D(256, 3, padding='same', activation=None),
        keras.layers.BatchNormalization(),
        keras.layers.Add(),
        keras.layers.Activation('relu'),
        keras.layers.Conv2D(256, 3, padding='same', activation='relu'),
        keras.layers.BatchNormalization(),
        keras.layers.Conv2D(256, 3, padding='same', activation=None),
        keras.layers.BatchNormalization(),
        keras.layers.Add(),
        keras.layers.Activation('relu'),
        keras.layers.Conv2D(256, 3, padding='same', activation='relu'),
        keras.layers.BatchNormalization(),
        keras.layers.Conv2D(256, 3, padding='same', activation=None),
        keras.layers.Conv2D(64, kernel_size=3, strides=2, padding="same", kernel_initializer=initializer, use_bias=False),
])


In [ ]:
# Define the generator models
generator_G = keras.Sequential([
    # Encoder
    keras.layers.Conv2D(64, kernel_size=4, strides=2, padding="same", input_shape=img_shape),
    keras.layers.LeakyReLU(alpha=0.2),
    keras.layers.Conv2D(128, kernel_size=4, strides=2, padding="same"),
    keras.layers.InstanceNormalization(),
    keras.layers.LeakyReLU(alpha=0.2),

    # Transformer
    keras.layers.Conv2D(256, kernel_size=4, strides=2, padding="same"),
    keras.layers.InstanceNormalization(),
    keras.layers.LeakyReLU(alpha=0.2),
    keras.layers.Conv2DTranspose(128, kernel_size=4, strides=2, padding="same"),
    keras.layers.InstanceNormalization(),
    keras.layers.LeakyReLU(alpha=0.2),

    # Decoder
    keras.layers.Conv2DTranspose(64, kernel_size=4, strides=2, padding="same"),
    keras.layers.InstanceNormalization(),
    keras.layers.LeakyReLU(alpha=0.2),
    keras.layers.Conv2DTranspose(3, kernel_size=4, strides=2, padding="same", activation="tanh")
])

generator_F = keras.Sequential([
    # Encoder
    keras.layers.Conv2D(64, kernel_size=4, strides=2, padding="same", input_shape=img_shape),
    keras.layers.LeakyReLU(alpha=0.2),
    keras.layers.Conv2D(128, kernel_size=4, strides=2, padding="same"),
    keras.layers.InstanceNormalization(),
    keras.layers.LeakyReLU(alpha=0.2),

    # Transformer
    keras.layers.Conv2D(256, kernel_size=4, strides=2, padding="same"),
    keras.layers.InstanceNormalization(),
    keras.layers.LeakyReLU(alpha=0.2),
    keras.layers.Conv2DTranspose(128, kernel_size=4, strides=2, padding="same"),
    keras.layers.InstanceNormalization(),
    keras.layers.LeakyReLU(alpha=0.2),

    # Decoder
    keras.layers.Conv2DTranspose(64, kernel_size=4, strides=2, padding="same"),
    keras.layers.InstanceNormalization(),
    keras.layers.LeakyReLU(alpha=0.2),
    keras.layers.Conv2DTranspose(3, kernel_size=4, strides=2, padding="same", activation="tanh")
])


In [ ]:
# Define the discriminator models
    # discriminator_X = keras.Sequential([...])
   # discriminator_Y = keras.Sequential([...])

discriminator_X = keras.Sequential([
    keras.layers.Conv2D(64, kernel_size=4, strides=2, padding="same", input_shape=img_shape),
    keras.layers.LeakyReLU(alpha=0.2),
    keras.layers.Conv2D(128, kernel_size=4, strides=2, padding="same"),
    keras.layers.InstanceNormalization(),
    keras.layers.LeakyReLU(alpha=0.2),
    keras.layers.Conv2D(256, kernel_size=4, strides=2, padding="same"),
    keras.layers.InstanceNormalization(),
    keras.layers.LeakyReLU(alpha=0.2),
    keras.layers.Conv2D(512, kernel_size=4, strides=2, padding="same"),
    keras.layers.InstanceNormalization(),
    keras.layers.LeakyReLU(alpha=0.2),
    keras.layers.Conv2D(1, kernel_size=4, strides=1, padding="same")
])

discriminator_Y = keras.Sequential([
    keras.layers.Conv2D(64, kernel_size=4, strides=2, padding="same", input_shape=img_shape),
    keras.layers.LeakyReLU(alpha=0.2),
    keras.layers.Conv2D(128, kernel_size=4, strides=2, padding="same"),
    keras.layers.InstanceNormalization(),
    keras.layers.LeakyReLU(alpha=0.2),
    keras.layers.Conv2D(256, kernel_size=4, strides=2, padding="same"),
    keras.layers.InstanceNormalization(),
    keras.layers.LeakyReLU(alpha=0.2),
    keras.layers.Conv2D(512, kernel_size=4, strides=2, padding="same"),
    keras.layers.InstanceNormalization(),
    keras.layers.LeakyReLU(alpha=0.2),
    keras.layers.Conv2D(1, kernel_size=4, strides=1, padding="same")
])


In [ ]:
# Define the loss functions for the Cycle-GAN algorithm. 
# The adversarial loss, cycle consistency loss, and identity loss should all be used.
# Define the adversarial loss function
def adversarial_loss(y_true, y_pred):
    return keras.losses.BinaryCrossentropy()(y_true, y_pred)

# Define the cycle consistency loss function
def cycle_consistency_loss(y_true, y_pred):
    return keras.losses.MeanAbsoluteError()(y_true, y_pred)

# Define the identity loss function
def identity_loss(y_true, y_pred):
    return keras.losses.MeanAbsoluteError()(y_true, y_pred)


# Train the Cycle-GAN algorithm using the preprocessed dataset.
# During training, the mean absolute and mean squared errors can be used as evaluation metrics.

In [ ]:

# Define the optimizers for the generator and discriminator models
optimizer = keras.optimizers.Adam(learning_rate=0.0002, beta_1=0.5)

# Compile the generator and discriminator models
generator_G.compile(loss=[adversarial_loss, cycle_consistency_loss, identity_loss], optimizer=optimizer)
generator_F.compile(loss=[adversarial_loss, cycle_consistency_loss, identity_loss], optimizer=optimizer)
discriminator_X.compile(loss='binary_crossentropy', optimizer=optimizer)
discriminator_Y.compile(loss='binary_crossentropy', optimizer=optimizer)

# Train the Cycle-GAN algorithm
for epoch in range(epochs):
    for batch in train_generator:
        # Get the batch of images and labels
        real_images_X, real_images_Y = batch
        
        # Train the generator models
        with tf.GradientTape(persistent=True) as tape:
            # Generate fake images
            fake_images_Y = generator_G(real_images_X, training=True)
            fake_images_X = generator_F(real_images_Y, training=True)
            
            # Reconstruct original images
            reconstructed_images_X = generator_F(fake_images_Y, training=True)
            reconstructed_images_Y = generator_G(fake_images_X, training=True)
            
            # Calculate the adversarial, cycle consistency, and identity losses
            adversarial_loss_X = adversarial_loss(tf.ones_like(discriminator_X(real_images_X)), discriminator_X(real_images_X))
            adversarial_loss_Y = adversarial_loss(tf.ones_like(discriminator_Y(real_images_Y)), discriminator_Y(real_images_Y))
            cycle_consistency_loss_X = cycle_consistency_loss(real_images_X, reconstructed_images_X)
            cycle_consistency_loss_Y = cycle_consistency_loss(real_images_Y, reconstructed_images_Y)
            identity_loss_X = identity_loss(real_images_X, generator_F(real_images_X))
            identity_loss_Y = identity_loss(real_images_Y, generator_G(real_images_Y))
            
            # Calculate the total generator losses
            total_loss_G = (
                adversarial_loss_X 
                + adversarial_loss_Y 
                + cycle_consistency_loss_X 
                + cycle_consistency_loss_Y 
                + identity_loss_X 
                + identity_loss_Y
            )
        
        # Calculate the gradients for the generator models
        gradients_G = tape.gradient(total_loss_G, generator_G.trainable_variables + generator_F.trainable_variables)
        
        # Apply the gradients to the generator models
        optimizer.apply_gradients(zip(gradients_G, generator_G.trainable_variables + generator_F.trainable_variables))
        
        # Train the discriminator models
        with tf.GradientTape(persistent=True) as tape:
            # Calculate the discriminator losses for real and fake images
            real_loss_X = tf.reduce_mean(tf.keras.losses.binary_crossentropy(tf.ones_like(discriminator_X(real_images_X)), discriminator_X(real_images_X)))
            fake_loss_X = tf.reduce_mean(tf.keras.losses.binary_crossentropy(tf.zeros_like(discriminator_X(fake_images_X)), discriminator_X(fake_images_X)))
            real_loss_Y = tf.reduce_mean(tf.keras.losses.binary_crossentropy(tf.ones_like(discriminator_Y(real_images_Y)), discriminator_Y(real_images_Y)))
            fake_loss_Y = tf.reduce_mean(tf.keras.losses.binary_crossentropy(tf.zeros_like(discriminator_Y(fake_images_Y)), discriminator_Y(fake_images_Y)))
            
  
           


In [ ]:
          # Calculate the total discriminator losses
# Define the lists to store the discriminator losses
disc_loss_X_real = []
disc_loss_X_fake = []
disc_loss_Y_real = []
disc_loss_Y_fake = []

# Train the discriminators
for i, batch in enumerate(train_dataset):
    
    
    # Train the discriminators
for i, batch in enumerate(train_dataset):
    # Get the real images
    real_X, real_Y = batch[0], batch[1]
    # Generate fake images
    fake_X = generator_F(real_Y)
    fake_Y = generator_G(real_X)
    # Train the discriminator for real and fake images
    discriminator_X.trainable = True
    discriminator_Y.trainable = True
    d_X_loss_real = discriminator_X.train_on_batch(real_X, np.ones((batch_size, 1)))
    d_X_loss_fake = discriminator_X.train_on_batch(fake_X, np.zeros((batch_size, 1)))
    d_X_loss = 0.5 * np.add(d_X_loss_real, d_X_loss_fake)
    d_Y_loss_real = discriminator_Y.train_on_batch(real_Y, np.ones((batch_size, 1)))
    d_Y_loss_fake = discriminator_Y.train_on_batch(fake_Y, np.zeros((batch_size, 1)))
    d_Y_loss = 0.5 * np.add(d_Y_loss_real, d_Y_loss_fake)
    # Update discriminator weights
    discriminator_X.trainable = False
    discriminator_Y.trainable = False
    # Train the generator
    g_loss = combined.train_on_batch([real_X, real_Y], [np.ones((batch_size, 1)), real_X, real_Y, real_X, real_Y])
    # Print the progress
    if i % print_interval == 0:
        print("Epoch {} Batch {} Discriminator_X_Loss {:.4f} Discriminator_Y_Loss {:.4f} Generator_Loss {:.4f}".format(
            epoch+1, i+1, d_X_loss, d_Y_loss, g_loss[0]))

   


In [ ]:
# # Calculate the discriminator losses for this batch
disc_loss_X_real.append(d_loss_X_real)
disc_loss_X_fake.append(d_loss_X_fake)
disc_loss_Y_real.append(d_loss_Y_real)
disc_loss_Y_fake.append(d_loss_Y_fake)

# Calculate the mean discriminator losses for this epoch
mean_disc_loss_X_real = np.mean(disc_loss_X_real)
mean_disc_loss_X_fake = np.mean(disc_loss_X_fake)
mean_disc_loss_Y_real = np.mean(disc_loss_Y_real)
mean_disc_loss_Y_fake = np.mean(disc_loss_Y_fake)
total_disc_loss = mean_disc_loss_X_real + mean_disc_loss_X_fake + mean_disc_loss_Y_real + mean_disc_loss_Y_fake

#  Evaluate the trained Cycle-GAN algorithm using mean absolute and mean squared errors:

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np

# Load the trained Cycle-GAN model
cycle_gan = tf.keras.models.load_model('cycle_gan_model.h5')

# Load the test dataset
test_data_dir = 'C:\\Users\\n\\Downloads\\FruitNetDataset'
test_data_generator = ImageDataGenerator(rescale=1./255)
test_generator = test_data_generator.flow_from_directory(
    test_data_dir,
    target_size=(256, 256),
    batch_size=32,
    class_mode=None,
    shuffle=False
)

# Generate fake images using the Cycle-GAN model
fake_images = cycle_gan.predict(test_generator)

# Calculate mean absolute error and mean squared error
mae = np.mean(np.abs(test_generator - fake_images))
mse = np.mean(np.square(test_generator - fake_images))

print(f'Mean Absolute Error: {mae}')
print(f'Mean Squared Error: {mse}')
